Monte Carlo Prediction Simulation

Black Jack Game or 21 Card Game

In [1]:
import gym
env=gym.make('Blackjack-v1')

In [2]:
print(env.reset())

((20, 10, False), {})


In [3]:
print(env.action_space)

Discrete(2)


In [4]:
import pandas as pd
from collections import defaultdict

In [5]:
state,empty=env.reset()
print(state)

(19, 6, False)


In [6]:
valu=state[0]

def policy(state):
    return 0 if valu > 19 else 1

In [7]:
print(valu)

19


In [8]:
print(policy(state))

1


Generating Episode

In [9]:
num_timesteps = 100
def generate_episode(policy):
    episode=[]
    state=env.reset()[0]
    for t in range(num_timesteps):
        action=policy(state)
        next_state,reward,done,info,empt = env.step(action)
        episode.append((state,action,reward))
        if done:
            break
        state=next_state
    return episode

In [10]:
print(generate_episode(policy))

[((19, 3, True), 1, 0.0), ((19, 3, False), 1, -1.0)]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Computing Value Function


In [11]:
total_return=defaultdict(float)
N=defaultdict(int)
num_iterations = 500000
for i in range (num_iterations):
    episode=generate_episode(policy)
    states,actions,rewards=zip(*episode)
    for t, state in enumerate(states):
        R=(sum(rewards[t:]))
        total_return[state]= total_return[state]+R
        N[state] = N[state]+ 1
total_return=pd.DataFrame(total_return.items(),columns=['state','total_return'])
N= pd.DataFrame(N.items(),columns=['state','N'])    
df= pd.merge(total_return,N,on="state")
df.head(10)   


,state,total_return,N
0,"(12, 6, False)",-4743.0,4743
1,"(15, 7, False)",-5339.0,5339
2,"(19, 7, False)",-6089.0,6089
3,"(5, 10, False)",-1757.0,1757
4,"(10, 10, False)",-7523.0,7523
5,"(12, 10, False)",-19325.0,19325
6,"(10, 3, False)",-1860.0,1860
7,"(20, 3, False)",-8174.0,8174
8,"(13, 4, False)",-5015.0,5015
9,"(18, 4, False)",-5763.0,5763


In [12]:
print(total_return)
print(N)

               state  total_return
0     (12, 6, False)       -4743.0
1     (15, 7, False)       -5339.0
2     (19, 7, False)       -6089.0
3     (5, 10, False)       -1757.0
4    (10, 10, False)       -7523.0
..               ...           ...
275    (12, 2, True)        -228.0
276    (4, 2, False)        -217.0
277    (4, 9, False)        -235.0
278    (13, 6, True)        -482.0
279    (15, 9, True)        -581.0

[280 rows x 2 columns]
               state     N
0     (12, 6, False)  4743
1     (15, 7, False)  5339
2     (19, 7, False)  6089
3     (5, 10, False)  1757
4    (10, 10, False)  7523
..               ...   ...
275    (12, 2, True)   228
276    (4, 2, False)   217
277    (4, 9, False)   235
278    (13, 6, True)   482
279    (15, 9, True)   581

[280 rows x 2 columns]


In [13]:
states,actions,rewards=zip(*episode)
print(states)
print(actions)
print(rewards)


((18, 2, False), (21, 2, False))
(1, 1)
(0.0, -1.0)


In [14]:
df['value']= df['total_return']/df['N']
df.head(10)

,state,total_return,N,value
0,"(12, 6, False)",-4743.0,4743,-1.0
1,"(15, 7, False)",-5339.0,5339,-1.0
2,"(19, 7, False)",-6089.0,6089,-1.0
3,"(5, 10, False)",-1757.0,1757,-1.0
4,"(10, 10, False)",-7523.0,7523,-1.0
5,"(12, 10, False)",-19325.0,19325,-1.0
6,"(10, 3, False)",-1860.0,1860,-1.0
7,"(20, 3, False)",-8174.0,8174,-1.0
8,"(13, 4, False)",-5015.0,5015,-1.0
9,"(18, 4, False)",-5763.0,5763,-1.0


In [15]:
df[df['state']==(21,9,False)]['value'].values

array([-1.])

In [16]:
df[df['state']==(5,8,False)]['value'].values

array([-1.])